# EX 1
### Scrape an article from ProPubblica

Define a function that takes a url of an article from the ProPubblica website and extracts:
  - the title
  - the "abstract", i.e: the text between the name of the author and the title 
  - the author
  - the publication date
  - the text in the main body
  
As usual, the output is a dictionary. Use the element inspector in your web console to get the right information. All the html elements can be retrieved using the selenium.get() and the BeautifulSoup() methods. 

![ProPublica](../img/propublica.png)

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(executable_path= "chromedriver.exe")
sample_article = "https://www.propublica.org/article/dr-death-christopher-duntsch-a-surgeon-so-bad-it-was-criminal"
art = "https://www.propublica.org/article/q-and-a-can-a-divided-europe-handle-the-refugee-crisis"

In [ ]:
# Complete this function

def juice_pro_publica(a_url):
    """Insert description here"""
    source = ""
    soup = ""
    d = {}
    d["Title"] = ""
    d["Abstract"] = "" 
    d["Author"] = ""
    d["Time"] = ""
    d["text"] = []
    
    
    return d

In [ ]:
def juice_pro_publica(driver, url):

    driver.get(url)
    print(f'Waiting 5 seconds before scraping the article\n>>> {url}')
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source)
    d = {}
    try: 
        header = soup.find("header", class_="article-header")
        article = soup.find("div", class_="article-wrap")

        title = header.find("h2", class_="hed").get_text()
        try: 
            abstract = header.find("p", class_="dek").get_text()
        except:
            abstract = None
        try:
            author = header.find("span", class_="name").get_text()
        except:
            author = header.find("a", class_="name").get_text()

        date = header.find("time", class_="timestamp").get_text()
        text_article = [p.get_text().strip() for p in article.find_all("p")]

        d["Title"] = title
        d["Abstract"] = abstract 
        d["Author"] = author
        d["Time"] = date
        d["text"] = text_article
            
    except AttributeError as e:
        print(f"ERROR in {url}: problems in scraping all the info!")
        error = f'error in {url}'
    
    return d

In [ ]:
juice_pro_publica(driver, sample_article)

# Ex 2

On the ProPublica website it is possible to search for all articles related to a search key. Try to do several searches by changing the key each time and see how the url changes. Then write a function that takes a search key as an argument and returns a list of all the related links shown on the results page.


Search button:

![ProPublica2](../img/propublica_2.png)

Results page:

![ProPublica3](../img/propublica_3.png)

In [ ]:
import time

def get_hrefs_to_articles(driver,key):
    
    """Takes a word, points the driver to the propublica webpage that contains the search 
    results for that word, collects the hrefs from the results   """
    
    base_url = "https://www.propublica.org/search?qss=" + key.replace(" ", "+")
    driver.get(base_url)
    # New line in the function: it forces Python to wait before executing the next lines of code
    print(f'Waiting 5 seconds before scraping all the links for the key "{key}"')
    time.sleep(5)
    rs =BeautifulSoup(driver.page_source).find_all("a", class_="gs-title")
    out_= []
    for element in rs:
        try:
#           if re.search("""^https://www.propublica.org/article/""", element["href"]):  --> regex version  
            if element["href"].startswith("https://www.propublica.org/article/"):
                out_.append(element["href"])
        except KeyError:
            pass
    out_ = list(set(out_))        
    return out_

In [ ]:
get_hrefs_to_articles(driver,"italy")

# Ex 3

Combine the functions of the two previous exercises. Write a function that, given a search key, is able to extract and save in a dictionary the information (title, abstract, author, date, and text body) of the articles related to the search key. 

In [ ]:
key = "italy"

def get_all_articles(driver,key_):
    links = get_hrefs_to_articles(driver,key)
    dataset = {key : []}
    for url in links:
        dataset[key].append(juice_pro_publica(driver,url))
    return dataset

In [ ]:
get_all_articles(driver,key)

# Ex 4

Let's make things even more complicated.Write a function that can extract the information (title, abstract, author, date, and text body) contained in the articles related to each of the search keys in the list below.  Save all the information in a dictionary. 

In [ ]:
keys = ["italy", "Apple", "Twitter", "Obama", "Bill Gates"]

dataset = {}
for key in keys:
    d = get_all_articles(driver,key)
    dataset.update(d)